# Assignment 3
## Econ 8310 - Business Forecasting

For homework assignment 3, you will work with [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), a more fancier data set.

- You must create a custom data loader as described in the first week of neural network lectures [2 points]
    - You will NOT receive credit for this if you use the pytorch prebuilt loader for Fashion MNIST!
- You must create a working and trained neural network using only pytorch [2 points]
- You must store your weights and create an import script so that I can evaluate your model without training it [2 points]

Highest accuracy score gets some extra credit!

Submit your forked repository URL on Canvas! :) I'll be manually grading this assignment.

Some checks you can make on your own:
- Did you manually process the data or use a prebuilt loader (see above)?
- Does your script train a neural network on the assigned data?
- Did your script save your model?
- Do you have separate code to import your model for use after training?

In [1]:
import gzip
import numpy as np
import torch

def read_images(filename):
    with gzip.open(filename, 'rb') as f:
        f.read(16)  # header skipping
        buffer = f.read()
        data = np.frombuffer(buffer, dtype=np.uint8).astype(np.float32)
        data = data.reshape(-1, 28*28) / 255.0  # Normalize
        return torch.tensor(data)

def read_labels(filename):
    with gzip.open(filename, 'rb') as f:
        f.read(8)
        buffer = f.read()
        labels = np.frombuffer(buffer, dtype=np.uint8)
        return torch.tensor(labels, dtype=torch.long)

def get_fashion_mnist_data(path='.'):
    train_x = read_images(f'{path}/train-images-idx3-ubyte.gz')
    train_y = read_labels(f'{path}/train-labels-idx1-ubyte.gz')
    test_x = read_images(f'{path}/t10k-images-idx3-ubyte.gz')
    test_y = read_labels(f'{path}/t10k-labels-idx1-ubyte.gz')
    return train_x, train_y, test_x, test_y

# Load and verify the data
train_x, train_y, test_x, test_y = get_fashion_mnist_data()
print("Train:", train_x.shape, train_y.shape)
print("Test :", test_x.shape, test_y.shape)


Train: torch.Size([60000, 784]) torch.Size([60000])
Test : torch.Size([10000, 784]) torch.Size([10000])


In [24]:
import torch
import torch.nn as nn

class FashionClassifier(nn.Module):
    def __init__(self):
        super(FashionClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(784, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)


In [25]:
import torch.nn.functional as F
from torch import optim

model = FashionClassifier()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

batch_size = 64
epochs = 50

for epoch in range(epochs):
    permutation = torch.randperm(train_x.size(0))
    epoch_loss = 0
    correct = 0
    total = 0

    for i in range(0, train_x.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        x_batch = train_x[indices]
        y_batch = train_y[indices]

        preds = model(x_batch)
        loss = F.cross_entropy(preds, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        correct += (preds.argmax(1) == y_batch).sum().item()
        total += y_batch.size(0)

    scheduler.step()
    train_acc = correct / total
    print(f"Epoch {epoch+1:2d} | Loss: {epoch_loss:.4f} | Train Acc: {train_acc:.4f}")


Epoch  1 | Loss: 492.0804 | Train Acc: 0.8199
Epoch  2 | Loss: 377.4563 | Train Acc: 0.8538
Epoch  3 | Loss: 349.2116 | Train Acc: 0.8651
Epoch  4 | Loss: 323.9655 | Train Acc: 0.8730
Epoch  5 | Loss: 310.7680 | Train Acc: 0.8780
Epoch  6 | Loss: 298.9636 | Train Acc: 0.8815
Epoch  7 | Loss: 287.5709 | Train Acc: 0.8867
Epoch  8 | Loss: 277.0797 | Train Acc: 0.8901
Epoch  9 | Loss: 268.5758 | Train Acc: 0.8938
Epoch 10 | Loss: 264.0986 | Train Acc: 0.8952
Epoch 11 | Loss: 236.4331 | Train Acc: 0.9046
Epoch 12 | Loss: 228.8722 | Train Acc: 0.9095
Epoch 13 | Loss: 222.9869 | Train Acc: 0.9120
Epoch 14 | Loss: 219.4331 | Train Acc: 0.9125
Epoch 15 | Loss: 214.0352 | Train Acc: 0.9153
Epoch 16 | Loss: 210.4927 | Train Acc: 0.9153
Epoch 17 | Loss: 205.1685 | Train Acc: 0.9176
Epoch 18 | Loss: 200.7757 | Train Acc: 0.9200
Epoch 19 | Loss: 197.9303 | Train Acc: 0.9212
Epoch 20 | Loss: 195.9485 | Train Acc: 0.9208
Epoch 21 | Loss: 181.0443 | Train Acc: 0.9271
Epoch 22 | Loss: 176.0588 | Train 

In [26]:
torch.save(model.state_dict(), 'fashion_model_v2.pth')

In [27]:
model = FashionClassifier()
model.load_state_dict(torch.load('fashion_model_v2.pth'))
model.eval()

with torch.no_grad():
    outputs = model(test_x)
    predicted = outputs.argmax(dim=1)
    accuracy = (predicted == test_y).float().mean()

print(f" Final Test Accuracy: {accuracy:.4f}")


 Final Test Accuracy: 0.9050


In [32]:
%%writefile model.py
import torch.nn as nn

class FashionClassifier(nn.Module):
    def __init__(self):
        super(FashionClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(784, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)


Overwriting model.py
